In [1]:
from yggdrasil.examples import yamls as ex_yamls
import yaml
import jupyterlab_nodeeditor as jlne

In [8]:
# Improved version of making a JLNE-compliant dictionary from a Yggdrasil Model YAML

def dict_conversion(model_file):
    # Using flags here in case I need them later, will remove if I don't
    is_dict, is_list = False, False
    
    # If the type of the safe load is a dictionary, we know it is a single model
    # If it is a list, then we have multiple models
    # If it's neither, return the type so I can work on it
    if type(model_file) is dict:
        is_dict = True
    elif type(model_file) is list:
        is_list = True
    else:
        return ("Unaccounted model type: ", type(model_file))
    
    # Let's do everything all at once, can split it up if needed
    # Check if it's a single model (dictionary) or multiple (list)
    if is_dict:
        # Setup initial dictionary to be filled
        new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], []
        new_dict["title"] = model_file["name"]
        
        # Fill in the Inputs
        if "input" in model_file.keys():
            for i, inp in enumerate(model_file["input"]):
                new_dict["inputs"].append({'title': inp["name"], 'key': f"temp_in{i}", 'socket_type': inp["default_file"]["filetype"]})
        elif "inputs" in model_file.keys():
            for i, inp in enumerate(model_file["inputs"]):
                new_dict["inputs"].append({'title': inp["name"], 'key': f"temp_in{i}", 'socket_type': inp["default_file"]["filetype"]})
        
        # Fill in the Outputs, same as inputs with name changes
        if "output" in model_file.keys():
            for o, out in enumerate(model_file["output"]):
                new_dict["outputs"].append({'title': out["name"], 'key': f"temp_out{o}", 'socket_type': out["default_file"]["filetype"]})
        elif "outputs" in model_file.keys():
            for o, out in enumerate(model_file["outputs"]):
                new_dict["outputs"].append({'title': out["name"], 'key': f"temp_out{o}", 'socket_type': out["default_file"]["filetype"]})
        
        # We return a list here so that way loading the models is uniform for both multi-model and single-model files
        return [new_dict]
    

    # We can pretty much do the dictionary loop multiple times per model then return the list with minor adjustments noted below
    elif is_list:
        model_list = []
        for model in model_file:
            # Setup initial dictionary to be filled
            new_dict, new_dict["inputs"], new_dict["outputs"],  = {}, [], []
            new_dict["title"] = model["name"]
        
            # Fill in the Inputs, no enumeration needed here
            # TO-DO For some reason, there is a list with a dictionary inside of it here, can fix this later
            # Matt can probably help with this issue
            if "input" in model.keys():
                if "type" in model["input"][0]:
                    new_dict["inputs"].append({'title': model["input"][0]["name"], 'key': "temp_in", 'socket_type': model["input"][0]["type"]})
                else:
                    new_dict["inputs"].append({'title': model["input"], 'key': "temp_in", 'socket_type': "temp_socket"})
            elif "inputs" in model.keys():
                if "type" in model["inputs"][0]:
                    new_dict["inputs"].append({'title': model["inputs"][0]["name"], 'key': "temp_in", 'socket_type': model["inputs"][0]["type"]})
                else:
                    new_dict["inputs"].append({'title': model["inputs"], 'key': "temp_in", 'socket_type': "temp_socket"})
        
            # Fill in the Outputs, same as inputs with name changes
            if "output" in model.keys():
                if "type" in model["output"][0]:
                    new_dict["outputs"].append({'title': model["output"][0]["name"], 'key': "temp_in", 'socket_type': model["output"][0]["type"]})
                else:
                    new_dict["outputs"].append({'title': model["output"], 'key': "temp_in", 'socket_type': "temp_socket"})
            elif "outputs" in model.keys():
                if "type" in model["outputs"][0]:
                    new_dict["outputs"].append({'title': model["outputs"][0]["name"], 'key': "temp_in", 'socket_type': model["outputs"][0]["type"]})
                else:
                    new_dict["outputs"].append({'title': model["outputs"], 'key': "temp_in", 'socket_type': "temp_socket"})
                    
            model_list.append(new_dict)
        
        # Return the full converted list of models
        return model_list
    
def extract_socket_types(model):
    socket_types = set()
    for inout in ("inputs", "outputs"):
        if inout in model:
            for inoutmodel in model[inout]:
                socket_types.update((inoutmodel["socket_type"],))
    return tuple(socket_types)

In [3]:
# By default, load the photosynthesis model
# ps - Node Editor instance that it is added to, default blank
def load_model(ps = None):
    # E.g. filepath: sample_models/sample_1.yml
    filepath = input("Please enter filepath for model or leave blank for an example: ")
    
    # Variable that will be used as the processed model
    fsample = None
    
    # Check to see if input is blank or not, run the respective code to get the model from file or the Photosynthesis example
    if filepath:
        with open(filepath, "r") as sample:
            sample_model = yaml.safe_load(sample)
            if "model" in sample_model.keys():
                fsample = sample_model["model"]
            elif "models" in sample_model.keys():
                fsample = sample_model["models"]
    else:
        print("Loading Photosynthesis Model into Editor...")
        with open(ex_yamls['fakeplant']['python'], "r") as sample:
            fsample = yaml.safe_load(sample)["model"]
    
    # Convert it all and add it into the editor, then return the entire editor
    ne_instance = ps or jlne.NodeEditor()
    model_list = dict_conversion(fsample)    
    for model in model_list:
        socket_types = extract_socket_types(model)
        ne_instance.add_component(model)
    
    return ne_instance

In [4]:
with open(ex_yamls['fakeplant']['python'], "r") as sample:
    fsample = yaml.safe_load(sample)["model"]

In [32]:
model_list = dict_conversion(fsample)

In [33]:
model_list

[{'inputs': [{'title': 'light_intensity',
    'key': 'temp_in0',
    'socket_type': 'table'},
   {'title': 'temperature', 'key': 'temp_in1', 'socket_type': 'table'},
   {'title': 'co2', 'key': 'temp_in2', 'socket_type': 'table'}],
  'outputs': [{'title': 'photosynthesis_rate',
    'key': 'temp_out0',
    'socket_type': 'table'}],
  'title': 'PhotosynthesisModel'}]

In [35]:
stypes = extract_socket_types(model_list[0])
stypes = jlne.SocketCollection(socket_types=stypes)
model_list[0]["sockets"] = stypes
model_list[0]


{'inputs': [{'title': 'light_intensity',
   'key': 'temp_in0',
   'socket_type': 'table'},
  {'title': 'temperature', 'key': 'temp_in1', 'socket_type': 'table'},
  {'title': 'co2', 'key': 'temp_in2', 'socket_type': 'table'}],
 'outputs': [{'title': 'photosynthesis_rate',
   'key': 'temp_out0',
   'socket_type': 'table'}],
 'title': 'PhotosynthesisModel',
 'sockets': SocketCollection(socket_types=('table',))}

In [36]:
ne = jlne.NodeEditor()
ne.add_component(model_list[0])

In [30]:
ne

## Visual check
Look at the model in the JLNE instance and ensure that it has all components that can connect.

In [1]:
from jupyterlab_nodeeditor import ygg_support
s = ygg_support.load_yggdrasil_sample()
s.components[0]

{'inputs': [{'title': 'light_intensity',
   'key': 'temp_in_0',
   'socket_type': 'table'},
  {'title': 'temperature', 'key': 'temp_in_1', 'socket_type': 'table'},
  {'title': 'co2', 'key': 'temp_in_2', 'socket_type': 'table'}],
 'outputs': [{'title': 'photosynthesis_rate',
   'key': 'temp_in_0',
   'socket_type': 'table'}],
 'title': 'PhotosynthesisModel'}

In [2]:
s.sockets

('table',)

In [3]:
import jupyterlab_nodeeditor as jlne

In [4]:
ne = jlne.NodeEditor(socket_types=s.sockets)

In [5]:
ne.add_component(s.components[0])

In [6]:
ne

In [5]:
ne